Model 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SPAM text message 20170820 - Data.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
category = pd.get_dummies(df.Category)
df_catnew = pd.concat([df, category], axis=1)
df_catnew = df_catnew.drop(columns='Category')
df_catnew

,Message,ham,spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,0,1
5568,Will ü b going to esplanade fr home?,1,0
5569,"Pity, * was in mood for that. So...any other s...",1,0
5570,The guy did some bitching but I acted like i'd...,1,0


In [ ]:
meword = df_catnew['Message'].values
label = df_catnew[['ham', 'spam']].values

In [ ]:
meword_train, meword_test, label_train, label_test = train_test_split(meword, label, test_size=0.2)

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(meword_train) 
tokenizer.fit_on_texts(meword_test)

In [ ]:
sqns_train = tokenizer.texts_to_sequences(meword_train)
sqns_test = tokenizer.texts_to_sequences(meword_test)

In [ ]:
padded_train = pad_sequences(sqns_train) 
padded_test = pad_sequences(sqns_test)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 25
history = model.fit(padded_train, label_train, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/25
140/140 - 18s - loss: 0.2255 - accuracy: 0.9235 - val_loss: 0.0388 - val_accuracy: 0.9901
Epoch 2/25
140/140 - 15s - loss: 0.0341 - accuracy: 0.9908 - val_loss: 0.0385 - val_accuracy: 0.9892
Epoch 3/25
140/140 - 14s - loss: 0.0173 - accuracy: 0.9953 - val_loss: 0.0371 - val_accuracy: 0.9883
Epoch 4/25
140/140 - 15s - loss: 0.0090 - accuracy: 0.9973 - val_loss: 0.0316 - val_accuracy: 0.9919
Epoch 5/25
140/140 - 15s - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.0398 - val_accuracy: 0.9865
Epoch 6/25
140/140 - 15s - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0375 - val_accuracy: 0.9928
Epoch 7/25
140/140 - 15s - loss: 4.7536e-04 - accuracy: 1.0000 - val_loss: 0.0520 - val_accuracy: 0.9865
Epoch 8/25
140/140 - 15s - loss: 0.0035 - accuracy: 0.9984 - val_loss: 0.0370 - val_accuracy: 0.9928
Epoch 9/25
140/140 - 15s - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0408 - val_accuracy: 0.9892
Epoch 10/25
140/140 - 15s - loss: 2.0565e-04 - accuracy: 1.0000 - val_loss: 0.0377 - va

KONVERSI KE TfLITE


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4hdw5gfn/assets


INFO:tensorflow:Assets written to: /tmp/tmp4hdw5gfn/assets


In [ ]:
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)